In [8]:
###### INCLUSION CRITERIA ANALYSIS###
####The below code 
#Creators: Xiaoru Dong
#Contributors:
#Last updated: 12/10/2018

In [4]:
import pandas as pd
from collections import OrderedDict

In [38]:
# Assign spreadsheet filename to `file`
file = '/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/Annotations_All.xlsx'

# Load spreadsheet
xl = pd.ExcelFile(file)

# Load a sheet into a DataFrame by name: df1
df = xl.parse('All Inclusion Criteria (5420)')

In [39]:
# The list of stop words
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
              "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
              'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 
              'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
              'the', 'if', 'because', 'as', 'until', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into',
              'through', 'during', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
              'under', 'again', 'further', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'few', 'more', 
              'most', 'some', 'such', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 
              'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain']

In [40]:
lineNum = len(df)

In [41]:
# set a function for outputing features
def writefeatures(path, features):
    with open(path,"w") as fp: 
        for word in features:
            fp.write(word + "\n")

In [42]:
# get an arff file used as the input in Weka
def writewekainput(path, features, lineNum, eachline, rct):
    with open(path,"w") as fp:        #this line of code to create a new file
        fp.write('''@RELATION classification_analysis\n''')
    
        for i in range(len(features)): 
            fp.write('''@ATTRIBUTE ''' + "word" + str(i) + ''' NUMERIC\n''') 
    
        fp.write('''@ATTRIBUTE class {1.0, 0.0}\n''')
        fp.write('''@DATA\n''')

    #this code is to check if a word appear in the feature list or not. If yes, then write "1", if no then write "0"
    
        for i in range(lineNum): 
            binary = {}
            for word in features:
                if word in eachline[i]:
                    binary[word] = 1
                else:
                    binary[word] = 0
                fp.write(str(binary[word]) + ",")
            fp.write(rct[i] + "\n")

In [43]:
# Extract Features

In [48]:
results1 = []
wordlist = []
rct = []
eachline = []

# split the sentence into words and save in a list
for i in range(lineNum): 
    
    string_rct = str(df['Only RCTs'][i])
    if string_rct == 'nan':
        string_rct = '0.0'
    if string_rct == 'x':
        string_rct = '1.0'
    rct.append(string_rct)
    
    string = str(df['Inclusion Criteria'][i]).lower().strip()
    wordlist = string.split(' ')
    
    for j in range(len(wordlist)):
        wordlist[j] = wordlist[j].replace('.', '')
        wordlist[j] = wordlist[j].replace(')', '')
        wordlist[j] = wordlist[j].replace('(', '')
        wordlist[j] = wordlist[j].replace(',', '')
        wordlist[j] = wordlist[j].replace("'", '')
        wordlist[j] = wordlist[j].replace('"', '')
        results1.append(wordlist[j])
    eachline.append(wordlist)

In [49]:
#remove empty element in the list
results1 = list(OrderedDict.fromkeys(results1))
results1 = [x for x in results1 if x] 
len(results1)

7292

In [50]:
# remove stop words
results1 = [word for word in results1 if word not in stop_words]
len(results1)

7202

In [51]:
# output features
featureword0_savepath = "/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/word_bag_0.txt"
writefeatures(featureword0_savepath, results1)

In [56]:
# output the weka input file
weka0_path="/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/classification_fall_weka_input_0.arff"
writewekainput(weka0_path, results1, lineNum, eachline, rct)

In [53]:
# Extract Features using information gain

In [74]:
# Assign spreadsheet filename to `file`
file = '/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/WekaResult/InfoWords.xlsx'

# Load spreadsheet
x2 = pd.ExcelFile(file)

# Load a sheet into a DataFrame by name: df2
df2 = x2.parse('InfoWords0')
df3 = x2.parse('InfoWords')

In [57]:
# read the informative words and save them in a list
info_wordlist = []
for word in (df2['Word Code in Weka']-1):
    info_wordlist.append(results1[word])

In [58]:
len(info_wordlist)

1662

In [59]:
# output features
featureinfoword0_savepath = "/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/informative_words_0.txt"
writefeatures(featureinfoword0_savepath, info_wordlist)

In [60]:
# output the weka input file
wekainfo0_path="/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/classification_fall_weka_input_info_0.arff"
writewekainput(wekainfo0_path, info_wordlist, lineNum, eachline, rct)

In [ ]:
# remove same meaning words

In [68]:
results2 = []
wordlist2 = []
rct = []
eachline2 = []

#replace the same meaning words
quasi = ['quasi-rcts', 'quasi-randomized', 'quasi-randomised', 'quasi-random', 'quasi-randomly']
cct = ['cct', 'ccts']
cba = ['cba', 'cbas', 'before-after']
privative = ["not","didn't", "don't", "won't", "isn't", "aren't", "wasn't", "weren't", "needn't", "wouldn't", "shouldn't", "can't", "couldn't", "no"]

# split the sentence into words and save in a list
for i in range(lineNum): 
    
    string_rct = str(df['Only RCTs'][i])
    if string_rct == 'nan':
        string_rct = '0.0'
    if string_rct == 'x':
        string_rct = '1.0'
    rct.append(string_rct)
    
    string = str(df['Inclusion Criteria'][i]).lower().strip()
    wordlist2 = string.split(' ')
    
    for j in range(len(wordlist2)):
        wordlist2[j] = wordlist2[j].replace('.', '')
        wordlist2[j] = wordlist2[j].replace(')', '')
        wordlist2[j] = wordlist2[j].replace('(', '')
        wordlist2[j] = wordlist2[j].replace(',', '')
        wordlist2[j] = wordlist2[j].replace("'", '')
        wordlist2[j] = wordlist2[j].replace('"', '')
        if wordlist2[j] in quasi:
            wordlist2[j] = 'quasi-rcts'
        if wordlist2[j] in cct:
            wordlist2[j] = 'cct'
        if wordlist2[j] in cba:
            wordlist2[j] = 'cba'
        if wordlist2[j] in privative:
            wordlist2[j] = 'not'
        results2.append(wordlist2[j])
    eachline2.append(wordlist2)

In [69]:
results2 = list(OrderedDict.fromkeys(results2))
results2 = [x for x in results2 if x] #remove empty element in the list
results2 = [word for word in results2 if word not in stop_words]
len(results2)

7194

In [70]:
# output features
featureinfoword_savepath = "/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/word_bag.txt"
writefeatures(featureinfoword_savepath, results2)

In [73]:
# output the weka input file
weka_path="/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/classification_fall_weka_input.arff"
writewekainput(weka_path, results2, lineNum, eachline2, rct)

In [14]:
# infomative words 

In [75]:
# read the informative words and save them in a list
info_wordlist = []
for word in (df3['Word Code in Weka']-1):
    info_wordlist.append(results2[word])

In [77]:
len(info_wordlist)

1655

In [78]:
# output features
featureinfoword_savepath = "/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/informative_words.txt"
writefeatures(featureinfoword_savepath, info_wordlist)

In [79]:
# output the weka input file
wekainfo_path="/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/classification_fall_weka_input_info.arff"
writewekainput(wekainfo_path, info_wordlist, lineNum, eachline2, rct)

In [ ]:
# Manual Analysis

In [80]:
# Assign spreadsheet filename to `file`
file = '/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/DataFile.xlsx'

# Load spreadsheet
x3 = pd.ExcelFile(file)

# Load a sheet into a DataFrame by name: df2
df4 = x3.parse('Informative Words2 (407)')

In [81]:
df4

,Word
0,quasi-rcts
1,birth
2,alternation
3,and
4,date
5,methods
6,cct
7,allocation
8,or
9,looking


In [82]:
# read the informative words and save them in a list
info_wordlist = []
for word in df4['Word']:
    info_wordlist.append(word)

In [84]:
len(info_wordlist)

407

In [85]:
# output the weka input file
wekainfo3_path="/Users/ruby/Desktop/Research/InclusionCriteriaClassification/Fall/InfoWord/classification_fall_weka_input_info_3.arff"
writewekainput(wekainfo3_path, info_wordlist, lineNum, eachline2, rct)